In [ ]:
# uncomment line below if using Google Colab
# !pip install evaluate

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import evaluate
import torch
from datasets import load_dataset, Dataset
from datasets import Dataset

In [ ]:
# load model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=7)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# load dataset
ds = load_dataset("dalopeza98/isear-cleaned-dataset")

In [ ]:
num_to_emotion = ["anger", "disgust", "fear", "guilt", "joy", "sadness", "shame"]

In [ ]:
# example listing of text and corresponding label
n = 74
text = ds["train"][n]["text"]
label = num_to_emotion[ds["train"][n]["label"]]
print(text)
print()
print(label)

I was drinking a glass of wine and eating something when there á
suddenly appeared pictures about the famine in some parts of á
Africa

shame


In [ ]:
def tokenize_function(examples):
    # tokenize and pad/truncate length
    tokenized_inputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

    return tokenized_inputs

# tokenize training and testing data
tokenized_train = ds["train"].map(tokenize_function)
tokenized_test = ds["test"].map(tokenize_function)

In [ ]:
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6027
})

In [ ]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy="epoch",
    report_to="none",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=100,
    gradient_accumulation_steps=1,
    num_train_epochs=3
)

'\ntraining_args = TrainingArguments(\n    output_dir=\'./results\',          # output directory\n    num_train_epochs=2,              # number of training epochs\n    per_device_train_batch_size=4,   # batch size for training\n    per_device_eval_batch_size=4,   # batch size for evaluation\n    warmup_steps=500,                # number of warmup steps for learning rate scheduler\n    weight_decay=0.01,               # strength of weight decay\n    logging_dir=\'./logs\',            # directory for storing logs\n    logging_steps=100,                # log every X updates\n    evaluation_strategy="epoch",     # evaluate at the end of every epoch\n    save_strategy="epoch",          # save model checkpoint every epoch\n    load_best_model_at_end=True,     # load the best model when finished training\n    report_to="none"\n)\n'

In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# trainer will store the fine-tuned model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

In [ ]:
eval_dataloader = trainer.get_eval_dataloader()
for batch in eval_dataloader:
    print(f"Batch size: {len(batch)}")  # Print the batch size of each batch
    break  # Only check the first batch

Batch size: 4


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.563600,1.494616,0.668879
2,0.604400,1.370502,0.691440
3,0.183800,1.774634,0.696085


TrainOutput(global_step=4521, training_loss=0.46495799474077026, metrics={'train_runtime': 1075.1319, 'train_samples_per_second': 16.817, 'train_steps_per_second': 4.205, 'total_flos': 2378762280921600.0, 'train_loss': 0.46495799474077026, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.774633526802063,
 'eval_accuracy': 0.6960849369608494,
 'eval_runtime': 23.7869,
 'eval_samples_per_second': 63.354,
 'eval_steps_per_second': 15.849,
 'epoch': 3.0}

In [ ]:
# confirm number of labels is 7
model_config = trainer.model.config
print(f"Number of labels: {model_config.num_labels}")

Number of labels: 7
Model type: bert


In [ ]:
index = 120

device = trainer.model.device  # ensure operations are being made on same device (should be GPU)

single_sample = tokenized_test[index]

# configured input as dictionary of tensors (expected by trainer)
inputs = {
    'input_ids': torch.tensor(single_sample['input_ids']).unsqueeze(0).to(device),
    'attention_mask': torch.tensor(single_sample['attention_mask']).unsqueeze(0).to(device),
}

if 'token_type_ids' in single_sample:
    inputs['token_type_ids'] = torch.tensor(single_sample['token_type_ids']).unsqueeze(0).to(device)

with torch.no_grad():
    logits = trainer.model(**inputs).logits

# convert logits to predicted class (the class with the highest logit)
predictions = np.argmax(logits.cpu().numpy(), axis=-1)

# store predicted class
predicted_class = predictions[0]


In [ ]:
print(f"Text: {single_sample['text']}")
print(f"Predicted class for the single sample: {num_to_emotion[predicted_class]}")

Text: I bought a Christmas present for my girl-friend. On my way home I á
was really glad about having found such a suitable thing
Predicted class for the single sample: joy
